In [1]:
import pandas as pd
file_path = "SubscriptionUseCase_Dataset.xlsx"
subs = pd.read_excel(file_path, sheet_name="Subscriptions")
plans = pd.read_excel(file_path, sheet_name="Subscription_Plans")
billing = pd.read_excel(file_path, sheet_name="Billing_Information")
#Step 1: Average spend per subscription
avg_spend = billing.groupby("subscription_id")["amount"].mean().reset_index(name="avg_spend")
# --- Step 2: Merge with subscriptions and plans ---
subs_small = subs[["Subscription Id", "User Id", "Product Id"]].rename(
    columns={"Subscription Id":"subscription_id", "Product Id":"product_id"}
)
plans_small = plans[["Product Id", "Price"]].rename(
    columns={"Product Id":"product_id", "Price":"plan_price"}
)

df = subs_small.merge(avg_spend, on="subscription_id", how="left") \
               .merge(plans_small, on="product_id", how="left")

df["avg_spend"] = df["avg_spend"].fillna(0)

# --- Step 3: Recommendation function ---
plans_sorted = plans_small.drop_duplicates().sort_values("plan_price")

def recommend_plan(avg_amount):
    candidates = plans_sorted[plans_sorted["plan_price"] >= avg_amount]
    if not candidates.empty:
        return candidates.iloc[0]["product_id"]
    else:
        return plans_sorted.iloc[-1]["product_id"]

df["recommended_plan_id"] = df["avg_spend"].apply(recommend_plan)
recommendations = df[["subscription_id", "User Id", "product_id", "avg_spend", "recommended_plan_id"]]

print(recommendations.head(10))


   subscription_id  User Id  product_id   avg_spend  recommended_plan_id
0                1       61          46  275.090000                 50.0
1                2       37          35    0.000000                 83.0
2                3       24          96   73.470000                 12.0
3                4       69          79  319.475000                 50.0
4                5        3          52  279.786667                 50.0
5                6       86          33  407.490000                 50.0
6                7       45          31   46.860000                 82.0
7                8       28          17  136.460000                 50.0
8                9       97          14    0.000000                 83.0
9               10       26          63  372.936667                 50.0


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
file_path = "SubscriptionUseCase_Dataset.xlsx"
subs = pd.read_excel(file_path, sheet_name="Subscriptions")
billing = pd.read_excel(file_path, sheet_name="Billing_Information")
# --- Step 1: Feature Engineering ---
renewal_count = billing[billing["payment_status"]=="paid"].groupby("subscription_id").size().reset_index(name="renewal_count")
avg_spend = billing.groupby("subscription_id")["amount"].mean().reset_index(name="avg_spend")
last_billed = billing.groupby("subscription_id")["billing_date"].max().reset_index(name="last_billed")
subs_small = subs[["Subscription Id", "User Id", "Start Date", "Terminated Date", "Status"]] \
              .rename(columns={"Subscription Id":"subscription_id"})

df = subs_small.merge(renewal_count, on="subscription_id", how="left") \
               .merge(avg_spend, on="subscription_id", how="left") \
               .merge(last_billed, on="subscription_id", how="left")

df[["renewal_count","avg_spend"]] = df[["renewal_count","avg_spend"]].fillna(0)
today = pd.to_datetime("2025-09-13")
df["Start Date"] = pd.to_datetime(df["Start Date"], errors="coerce")
df["Terminated Date"] = pd.to_datetime(df["Terminated Date"], errors="coerce")
df["end_date"] = df["Terminated Date"].fillna(df["last_billed"]).fillna(today)
df["duration_days"] = (df["end_date"] - df["Start Date"]).dt.days.clip(lower=0).fillna(0)

# --- Step 2: Labels ---
# Assume: active = Renew, otherwise = Churn
df["label"] = df["Status"].str.lower().apply(lambda x: "Renew" if x=="active" else "Churn")

# --- Step 3: Train Logistic Regression ---
X = df[["renewal_count","avg_spend","duration_days"]]
y = (df["label"]=="Renew").astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# --- Step 4: Predictions ---
df["predicted"] = model.predict(X)
df["predicted"] = df["predicted"].map({1:"Renew", 0:"Churn"})

# --- Step 5: Output ---
result = df[["subscription_id","User Id","renewal_count","avg_spend","duration_days","predicted"]]
print(result.head(10))


   subscription_id  User Id  renewal_count   avg_spend  duration_days  \
0                1       61            0.0  275.090000             86   
1                2       37            0.0    0.000000            385   
2                3       24            0.0   73.470000             39   
3                4       69            0.0  319.475000              9   
4                5        3            3.0  279.786667            109   
5                6       86            1.0  407.490000             44   
6                7       45            1.0   46.860000            176   
7                8       28            1.0  136.460000             12   
8                9       97            0.0    0.000000            529   
9               10       26            1.0  372.936667            224   

  predicted  
0     Churn  
1     Churn  
2     Renew  
3     Churn  
4     Renew  
5     Renew  
6     Renew  
7     Renew  
8     Churn  
9     Churn  
